In [1]:
import minsearch

In [2]:
import json

In [3]:
with open('documents.json','rt' ) as f_in :
    docs_raw = json.load(f_in)

In [4]:
documents = []

for course_dict in docs_raw :
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [7]:
index.fit(documents)

In [8]:
!pip install mistralai

In [9]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

In [14]:
import os

# Access the API key from the environment variable
api_key = os.environ['MISTRAL_AI_API_KEY']
model = "mistral-large-latest"

client = MistralClient(api_key=api_key)

chat_response = client.chat(
    model=model,
    messages=[ChatMessage(role="user", content=query)]
)

print(chat_response.choices[0].message.content)

Running Apache Kafka involves several steps, including downloading and configuring Kafka, starting the ZooKeeper server (which Kafka relies on), and then starting the Kafka server itself. Below are the detailed steps to get Kafka up and running:

### Prerequisites
1. **Java**: Ensure you have Java installed on your system. Kafka requires Java 8 or later.
2. **ZooKeeper**: Kafka relies on ZooKeeper for distributed coordination. ZooKeeper is typically bundled with Kafka, but you can also install it separately.

### Steps to Run Kafka

#### 1. Download Kafka
Download the latest version of Kafka from the [official website](https://kafka.apache.org/downloads).

```sh
wget https://downloads.apache.org/kafka/3.0.0/kafka_2.13-3.0.0.tgz
tar -xzf kafka_2.13-3.0.0.tgz
cd kafka_2.13-3.0.0
```

#### 2. Start ZooKeeper
Kafka comes with a ZooKeeper instance. Start ZooKeeper using the following command:

```sh
bin/zookeeper-server-start.sh config/zookeeper.properties
```

This command starts the ZooKe

In [15]:
def search (query):
    boost = {'question': 3.0, 'section' : 0.5}

    results = index.search(
    query=query,
    filter_dict={'course':'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=10
    )

    return results

In [21]:
def build_prompt (query, results) :
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: {context}

""".strip()

    context = ""
    
    for doc in results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [22]:
def llm(prompt):
    chat_response = client.chat(
    model=model,
    messages=[ChatMessage(role="user", content=prompt)]
    )

    return chat_response.choices[0].message.content

In [23]:
query = 'How do I run kafka?'

In [53]:
rag(query)

'Yes, you can still enroll in the course.'

In [54]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [55]:
from elasticsearch import Elasticsearch

In [56]:
es_client = Elasticsearch('http://localhost:9200')

In [57]:
index_settings = {
    
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/Xmeyoq6dST2IOZPmtdUytQ] already exists')

In [30]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [33]:
for doc in tqdm(documents):
 es_client.index(index=index_name, document=doc )

100%|██████████████████████████████████████████████| 948/948 [00:29<00:00, 31.89it/s]


In [42]:
query = "I just discovered the course. Can I still enroll?"

In [60]:
def elastice_search(query):

 search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
 }
 response = es_client.search(index=index_name, body=search_query)
 result_docs = []
 for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])
 return result_docs

In [61]:
elastice_search(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at